In [3]:
import boto3
import stepfunctions

from stepfunctions.steps import *
from stepfunctions.workflow import Workflow

In [9]:
LOCAL_PROFILE = 'ensamblador-isengard-dev'
REGION = 'us-east-1'

In [18]:
session = boto3.Session(profile_name=LOCAL_PROFILE, region_name=REGION)
sf_client = session.client('stepfunctions')

glue_job1_name = "litigantes_ddb_etl"
glue_job2_name = "remates_ddb_etl"
glue_job3_name = "documentos_ddb_etl"

In [29]:
workflow_execution_role= "arn:aws:iam::448716646650:role/AUCTIONS-INGEST-detailsstepmachineObtieneDetallesR-172G3CQ12F3NA"

job1_step  = GlueStartJobRunStep(glue_job1_name, parameters = { "JobName": glue_job1_name}  )
job2_step  = GlueStartJobRunStep(glue_job2_name, parameters = { "JobName": glue_job2_name}  )
job3_step  = GlueStartJobRunStep(glue_job3_name, parameters = {"JobName": glue_job3_name}  )

success_state = Pass(state_id="OK")
fail_state = Pass(state_id="ERROR")


first_stage = Parallel("primeros jobs")
first_stage.add_branch(job1_step)
first_stage.add_branch(job2_step)


my_choice_state = Choice ('Se ejecutó?')

my_choice_state.add_choice(
    rule=ChoiceRule.BooleanEquals(variable=job3_step.output()["Success"], value=True),
    next_step=success_state
)
my_choice_state.add_choice(
    ChoiceRule.BooleanEquals(variable=job3_step.output()["Success"], value=False),
    next_step=fail_state
)

basic_path = Chain([first_stage, job3_step, my_choice_state])


# Next, we define the workflow
basic_workflow = Workflow(
    name="glue_workflow", definition=basic_path, role=workflow_execution_role,
    client=sf_client
)

basic_workflow.render_graph()

In [30]:
basic_workflow.create()

'arn:aws:states:us-east-1:448716646650:stateMachine:glue_workflow'

In [16]:
#print(basic_workflow.definition.to_json(pretty=True))